# Coursera | Andrew Ng (05-week1)—循环序列模型

【第 5 部分-序列模型-第一周】在吴恩达深度学习视频基础上，笔记总结，添加个人理解。- ZJ
    
>[Coursera 课程](https://www.coursera.org/specializations/deep-learning) |[deeplearning.ai](https://www.deeplearning.ai/) |[网易云课堂](https://mooc.study.163.com/smartSpec/detail/1001319001.htm)

[CSDN](http://blog.csdn.net/JUNJUN_ZHAO/article/details/79362639)：http://blog.csdn.net/JUNJUN_ZHAO/article/details/79362639

---


# 循环序列模型 （Recurrent Neural Networks）


## <font color=#0099ff>1.1 为什么选择序列模型？(Why sequence models?)
    
深度学习中最激动人心的领域之一：**序列模型。**

例如**循环神经网络（RNN）**在语音识别，自然语言处理等其他领域中引起变革。

能够应用序列模型的例子:语音识别、音乐生成、情感分类、DNA 序列分析、机器翻译、视频活动识别、名称实体识别等。

![这里写图片描述](http://img.blog.csdn.net/20180224155604642?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)
   
**学习目标：学习构建自己的序列模型。**    
    
---
## <font color=#0099ff>1.2 数学符号 (Notation)
    
从**定义数学符号**开始，一步步构建序列模型。    

![这里写图片描述](http://img.blog.csdn.net/20180225121806505?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

- 输入 x : Harry Potter and Hermione Granger invented a new spell.

**目标：构建**一个能够**自动识别句中人名位置**的**序列模型**。---命名实体识别问题（Named-entity recognition）NER

此例子中为：识别  Harry Potter 和 Hermione Granger 

$x^{\langle 1 \rangle}  x^{\langle 2 \rangle}   x^{\langle 3 \rangle}   ......x^{\langle t \rangle} ......x^{\langle 9 \rangle} $   9 组特征表示上述 x 中 9 个单词。

- 输出 y : 1    1    0  1   1  0   0    0  0

$y^{\langle 1 \rangle}  y^{\langle 2 \rangle}   y^{\langle 3 \rangle}   ......y^{\langle t \rangle} ......y^{\langle 9 \rangle} $ 

t : 意味着这些是时间序列。

- 输入序列的长度：$T_x=9$

- 输出序列的长度：$T_y=9$

$x^{(i)\langle t \rangle}$: 第  i  个 训练样本 x 的第 t  个元素。 $T^{(i)}_x$ : 第 i 个训练样本的输入序列长度。

$y^{(i)\langle t \rangle}$: 第  i  个训练样本  y 的第 t  个元素。 $T^{(i)}_y$ : 第 i 个训练样本的输出序列长度。


![这里写图片描述](http://img.blog.csdn.net/20180225121824703?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)
   
 使用**自然语言处理 （NLP）**,**如何在序列中表示单个单词?**

首先，需要词汇表（字典）。列出在表示中所使用的所有单词。如图中所表示，不同的单词在自己的词汇表中对应得索引不同。
此例子中使用的 10000 个单词的词汇表，根据不同的商业模式，应用，所构建的词汇表不同，30000，50000，（大型互联网）1000000等。

**One-hot 向量表示法：**

如上图所示：$x^{\langle 1 \rangle}$ 是一个 10000 维度的向量，只有 第 4075 索引位置是 1，其余位置都是 0 。    
    
对于不在词汇表中的单词的表示：返回   `<UNK>`    (Unknown Word)

---
## <font color=#0099ff>1.3 循环神经网络模型 (Recurrent Neural Network Model)   
 
对于上述 命名实体识别问题 (NER) 之所以不使用标准神经网络来解决，是因为存在两个主要问题。

![这里写图片描述](http://img.blog.csdn.net/20180225153812706?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70) 

- 1.首先是在不同的例子中输入和输出可以是不同的长度。
- 2.像这样一个简单的神经网络架构，它不会共享在文本中不同位置上学到的特征。(例如卷积神经网络，可以传递学习到的特征等)
- 此外，每一个训练样本都是一个 10,000 维的向量。所以，这只是一个非常大的输入层，同时，第一层的权重矩阵将最终具有大量的参数。

 
![这里写图片描述](http://img.blog.csdn.net/20180225153839261?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

使用**循环神经网络（RNN）** 可以解决上述两个问题。

**什么是循环神经网络？**

如果你从左到右阅读句子，第一个单词 $x^{\langle 1 \rangle}$ 以及 初始化 $a^{\langle 0 \rangle}$ 零向量 ，带入到神经网络层（隐含层），尝试预测输出 $\hat{y}^{\langle 1 \rangle}$，同时在时间步 1 产生  $a^{\langle 1 \rangle}$。然后，将 $a^{\langle 1 \rangle}$ 和 $x^{\langle 2 \rangle}$带入到神经网络层（隐含层），尝试预测输出 $\hat{y}^{\langle 2 \rangle}$，然后前向广播，进入下一个时间步......直到最后输入 $x^{\langle T_x \rangle}$,输出 $\hat{y}^{\langle T_y \rangle}$。因此，在每个时间步，循环神经网络将这个激活传递给下一个时间步，供它使用。

在这个例子中，$T_x = T_y$ ,若 $T_x ， T_y$  不同，体系结构会改变一点。

上图中右侧图表，是在 论文或书籍中常见的图表，为了便于理解，Andrew Ng 使用左侧图表展开讲解。

每个时间步的参数是共享的，每一部分都受到参数的影响，如 $W_{aa},W_{ax},W_{ya}$,后面在解释这些参数的工作原理。

**RNN 的一个弱点：**，它只使用序列中较早的信息进行预测，特别是在预测  $\hat{y}^{\langle 3 \rangle}$ 时，不使用 $x^{\langle 4 \rangle}，x^{\langle 5 \rangle}，x^{\langle 6 \rangle}$ 等单词的信息。

例如： 

1. He said,"Teddy Roosevelt was a great President."
2. He said,"Teddy bears are on sale!"

上述例子中，Teddy Roosevelt 是人名。Teddy bears 并非我们想要识别的内容，但是如果只知道前三个单词 是无法准确识别的，还需要后面的单词信息。所以这个特定的神经网络结构的一个限制是，在某一时刻的预测使用输入或使用来自序列中较早的输入的信息，而不是在该序列的后面的信息。

后面的视频将讲解 **双向递归神经网络 （BRNN）**，来解决这个问题。

![这里写图片描述](http://img.blog.csdn.net/20180225153859865?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

**前向传播：** 

$$a^{\langle 0 \rangle}=\vec{0}$$

$$a^{\langle 1 \rangle}=g(W_{aa}a^{\langle 0 \rangle} + W_{ax}x^{\langle 1 \rangle} + b_a)$$ 

$$\hat{y}^{\langle 1 \rangle} = g(W_{ya}a^{\langle 1 \rangle} + b_y)$$

$$a^{\langle t \rangle}=g(W_{aa}a^{\langle t-1 \rangle} + W_{ax}x^{\langle t \rangle} + b_a)$$

$$\hat{y}^{\langle t \rangle} = g(W_{ya}a^{\langle t \rangle} + b_y)$$

对于激活函数计算 $a^{\langle t \rangle}$常用的是 **tanh** , 也有使用 **Relu** 的。

对于输出 y ,根据具体问题选取不同激活函数，如二分类问题可使用 **sigmoid**，其他 输出 如 softmax 等。

![这里写图片描述](http://img.blog.csdn.net/20180225153912301?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

**简化 RNN 符号表示：**

$$a^{\langle t \rangle}=g(W_{aa}a^{\langle t-1 \rangle} + W_{ax}x^{\langle t \rangle} + b_a)$$ 
$$\hat{y}^{\langle t \rangle} = g(W_{ya}a^{\langle t \rangle} + b_y)$$

简化为：

$$a^{\langle t \rangle}=g(W_{a}[a^{\langle t-1 \rangle} ,x^{\langle t \rangle} ]+ b_a)$$
$$\hat{y}^{\langle t \rangle} = g(W_{y}a^{\langle t \rangle} + b_y)$$   

使用矩阵的叠加简化，如 a 的维度是 100，x 的维度是 10000，那么 $W_{aa}$ 的维度是 （100，100），$W_{ax}$ 的维度是 （100，10000）,叠加后 $W_{a}$ 的维度是 （100，100100），如上图中所示。

    
---
## <font color=#0099ff>1.4 通过时间的反向传播  (Backpropogation through time)  
 
 上一节讲述的是 RNN 的基本结构，以及前向广播，这一节学习目标是：了解 RNN 的反向传播是如何工作的。
 
![这里写图片描述](http://img.blog.csdn.net/20180225180047463?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

红色箭头方向是反向传播。

![这里写图片描述](http://img.blog.csdn.net/20180225181207104?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)
    
如上图所示：红色箭头方向是反向传播。绿色线参数相同。

逻辑回归损失，也称为交叉熵损失。

单个单一位置或单个时间单一预测的单个词汇的损失：

$$L^{\langle t \rangle}(\hat{y}^{\langle t \rangle},y^{\langle t \rangle})= - y^{(i)}log\hat{y}^{\langle t \rangle} - (1- y^{\langle t \rangle}) log(1-\hat{y}^{\langle t \rangle} )$$

如上图所示，每个特征元素计算损失值。

整个序列的整体损失：

$$L(\hat{y},y)= \sum_{t=1}^{T_y} L^{\langle t \rangle}(\hat{y}^{\langle t \rangle},y^{\langle t \rangle})$$


以上是 RNN 前向和反向的工作原理，以上讲解的例子都是输入序列的长度等于输出序列的长度，接下来会展示更广泛的RNN架构。

---
## <font color=#0099ff>1.5 不同类型的循环神经网络  （Different types of RNNs）  

到目前为止，您已经看到了一种 RNN 架构，例如在 Name entity  recognition 中$T_x = T_y$。接下来介绍其他不同的 RNN 体系结构。
    
![这里写图片描述](http://img.blog.csdn.net/20180225184657976?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

上图所示例子中，展示了多种不同的 RNN 架构。$T_x 和 T_y$ 存在不相等的情况。

![这里写图片描述](http://img.blog.csdn.net/20180225184708713?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

- Many-to-Many(相等) : $T_x = T_y$ ,如 Name entity  recognition （命名实体识别）。

- Many-to-One :  $T_x \neq T_y$，$x = text, y = 0/1 或 1...5$ 等，如 Sentiment Classification (情感分类)，

- One-to-One : 标准神经网络。

![这里写图片描述](http://img.blog.csdn.net/20180225184718247?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)


- One-to- Many: $T_x \neq T_y$，$x = Ø$ ，x 可以为空集， 如 Music generation (音乐生成) 。当你真正生成序列的时候，经常会把这些前一个合成的输出文件输入到下一层。

- Many-to-Many(不等) : $T_x \neq T_y$ ，如 Machine translation (机器翻译)，法语翻译成英语，单词数量是不同的，则使前半部分为 encoder，共 $T_x$ 个，后半部分为 decoder  ,共 $T_y$ 个。

![这里写图片描述](http://img.blog.csdn.net/20180225184727814?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

其中很有意思的一部分是 序列生成，下节讲解。

---
## <font color=#0099ff>1.6 语言模型和序列生成  (Language  model and sequence generation)  
    
   那么什么是语言模型？
 
 ![这里写图片描述](http://img.blog.csdn.net/2018022610264353?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)   

在语音识别系统中，我说了一句话，然后一个语言模型能够告诉我，对于这句话识别得到的多种可能性中，哪种可能概率高。它所做的就是，可以告诉你某个特定的句子出现的概率是多少。

语言模型的基本工作原理可以理解为，输出一段语句，它可以给出每一部分的概率，表示为$P(y^{\langle 1 \rangle},y^{\langle 2 \rangle}......,y^{\langle T_y \rangle})$,注意，这里用 y 表示，比用 x 表示更合适。后面会解释。

![这里写图片描述](http://img.blog.csdn.net/20180226102652256?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

如何建立一个语言模型呢？要使用 RNN 建立这样一个模型，你首先需要一个训练集包括大量的英文文本。NLP 术语：语料库。

如上图所示，在有了以上基础的前提下，在训练集中的一个句子。Cats average 15  hours of sleep a day.

这句话结尾需添加一个 Tokenize `<EOS>`代表句子结束。 然后 每一个单词 使用 One-hot vectors 表示。如 10000 维度的向量。

如上图所示，不在词汇表中的使用 `<UNK>` 表示。

对于训练样本 $x^{\langle t \rangle} = y^{\langle t-1 \rangle}$, 下面解释为什么这么表示。


![这里写图片描述](http://img.blog.csdn.net/20180226102701203?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

如上图所示：注意一开始，从左向右看，第一个词是未知的。

$x^{\langle 1 \rangle}=\vec0$,$a^{\langle 0 \rangle}=\vec0$,然后预测输出 $\hat{y}^{\langle 1 \rangle}$ 使用 softmax ,假设词汇表是 10000 个，则预测输出的 概率是 `P(a)P(aron)....P(cats)...P(Zulu)` 额外还有 `P(<UNK>)P(<EOS>)` 共 10002 个。

然后在预测出第一个之后，会将第一个正确的单词作为输入，$x^{\langle 2 \rangle} = y ^{\langle 1 \rangle}$ 来预测第二个单词 $\hat{y}^{\langle 2 \rangle}$ , P(average | cats)。如上图所示，依次代入，一直到预测完最后一个单词。注意 因为句子结尾是 `<EOS>` 所以 最后一个的 输入为 $x^{\langle 9 \rangle} = y ^{\langle 8 \rangle}$ ，输出为 $\hat{y}^{\langle 9 \rangle}$ `P(<EOS>| ...... )`。 

单个语句的损失函数（softmax 层后的函数输出）：

$$L(\hat{y}^{\langle t \rangle},y^{\langle t \rangle}) = - \sum_{i}y_{i}^{\langle t \rangle}log\hat{y}_{i}^{\langle t \rangle}$$ 

整个句子的损失函数：

$$L = \sum L^{\langle t \rangle}(\hat{y}^{\langle t \rangle},y^{\langle t \rangle})$$

简单的例子，一个句子有三个单词，分别定义为 $y^{\langle 1 \rangle},y^{\langle 2 \rangle},y^{\langle 3 \rangle}$,那么它的概率是： 

$$P(y^{\langle 1 \rangle},y^{\langle 2 \rangle},y^{\langle 3 \rangle}) =P(y^{\langle 1 \rangle}) P(y^{\langle 2 \rangle} | y^{\langle 1 \rangle})P(y^{\langle 3 \rangle} | y^{\langle 1 \rangle},y^{\langle 2 \rangle})$$


---
## <font color=#0099ff>1.7 对新序列采样  （Sampling novel sequences）  

在训练完一个序列模型之后，要想了解这个模型学习到了什么，可以采用一个非正式的方法是：进行一次新序列采样。

记住：一个序列模型，模拟了任意特定单词序列的概率。

我们要做的就是对这个概率分布进行采样，来生成一个新的单词序列。 
    
![这里写图片描述](http://img.blog.csdn.net/20180226102731195?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)    

首先 ：$x^{\langle 1 \rangle}=\vec0$ , $a^{\langle 0 \rangle}=\vec0$, 然后预测输出 $\hat{y}^{\langle 1 \rangle}$ ，得出的概率比如是：P(a)P(aron)....P(cats)...P(Zulu)P(`<UNK>`) ,然后 使用采样函数，如 `np.random.choice` 对 softmax 分布进行随机采样。得出 $\hat{y}^{\langle 1 \rangle}$ 是 The, 然后输入到下一层，前向广播，进入下一个时间步。直到结束。

如何标记结束？有两种方式： 1. 词汇表中含有 `<EOS>`  2. 限定时间步，20 ，100 等。

而这个特定的程序有时会产生一个 `<UNK>` 的单词标记。如果你想确保你的算法从来没有产生这个标记，你可以做的一件事就是拒绝任何以未知的单词标记出来的样本，并且继续从其他词汇中重新采样，直到你得到一个不是未知的单词。

最后，经过以上步骤，就随机产生了一些序列语句。

![这里写图片描述](http://img.blog.csdn.net/2018022610274193?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

目前我们建立的都是基于词汇的 RNN 模型，根据实际应用，也可以建立一个基于字符的 RNN 结构。如上图所示。

Vocabulary = [a,b,c,...0,1,...9, ; ,', : , ,...A....Z]

相对应的  $y^{\langle 1 \rangle},y^{\langle 2 \rangle},y^{\langle 3 \rangle}$ 都是字符，而不是单个词汇，如 Cats average ....  $y^{\langle 1 \rangle}= C,y^{\langle 2 \rangle}=a,y^{\langle 3 \rangle}=t ,y^{\langle 4 \rangle}= 空格$等

基于字符的优点： 不用担心 UNK 出现。
主要的缺点：最后会得到太多太长的序列，所以在捕捉句子中的依赖关系方面，效果并不是很好。计算训练成本也很高。


![这里写图片描述](http://img.blog.csdn.net/2018022610275051?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)
    
上图，左侧是 一则新闻，右侧是 模仿莎士比亚的诗。

---
## <font color=#0099ff>1.8 带有神经网络的梯度消失  （Vanishing gradients with RNNs）  
    
前面已经了解了 RNN 的工作原理，接下来了解其中存在的问题，以及解决办法。

基本的 RNN 算法，存在一个很大的问题: 就是梯度消失。

![这里写图片描述](http://img.blog.csdn.net/20180226150729846?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

如图上所示：第一句 前面的 cat 单数 决定后面 是 was。 第二句 前面的 cats 复数决定后面是 were。这就是语言具有非常长期的依赖关系的一个例子。

而迄今为止我们已经看到了 RNN 的基础结构，它不是非常擅长捕获非常长期的依赖关系。

解释为什么？回想之前学过的梯度消失问题。

一个非常非常深的神经网络，前向传播，再反向传播，输出的 $\hat{y}$ 很难再影响前面层的权重。则 RNN 同理，梯度消失，不擅长捕捉长期依赖。

回想 梯度爆炸问题：指数级大的梯度 会导致 参数爆炸性增长，神经网络参数崩溃，会看到很多 NaN，这意味着在你的神经网络计算数值溢出的结果。

**总结：训练很深的神经网络时，随着层数的增加，导数会出现指数级的下降，则导致 梯度消失。或者指数级的增加，导致梯度爆炸。**

解决梯度爆炸问题可以使用 梯度修剪（gradient clipping）解决。观察梯度向量，如果大于某个阈值，可以适当地额缩放梯度，保证其不会太大，（通过最大值 阈值来修剪），相对具有鲁棒性。

而梯度消失就比较难解决，下面介绍 GRU （Gated Recurrent Unit）门控循环单元，来解决这个问题。


---
## <font color=#0099ff>1.9 GRU 单元  （Gated Recurrent Unit ）   
    

以上已经了解了基础 RNN 模型的运行机制。本节介绍 GRU 。改变了 神经网络隐藏层，使其可以更好的捕捉深层连接，并改善了梯度消失问题。

![这里写图片描述](http://img.blog.csdn.net/20180226150739302?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

$$a^{\langle t \rangle}=g(W_{a}[a^{\langle t-1 \rangle} ,x^{\langle t \rangle} ]+ b_a)$$ 

如计算在时间 t 的激活值 $a^{\langle t \rangle}$ , 输入  权重 $W_a$,上一个时间步的 激活值 $a^{\langle t-1 \rangle} $,当前输入值 $x^{\langle t \rangle}$,以及偏移量 $b_a$。激活函数 g ，比如是 tanh。 上图中 左侧部分，绘画了 输入及输出，经过 tanh  后，在经过 softmax ，再输出 $\hat{y}^{\langle t \rangle}$。

讲解这个图的目的是，将使用类似的图来讲解 GRU。

![这里写图片描述](http://img.blog.csdn.net/20180226154123228?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)


<font color=#0099ff>c = memory cell  : 记忆细胞，提供记忆能力，如 主语猫是单数还是复数。

- **$c^{\langle t \rangle}=a^{\langle t \rangle}$** 在 GRU 中这两个值是相同的，后面的 LSTM 中是不同的

- <font color=#0099ff>**$\tilde{c}^{\langle t \rangle}==tanh (W_c [c^{\langle t-1 \rangle},x^{\langle t \rangle}] +  b_c)$**

	$\tilde{c}^{\langle t \rangle}=$ : 候选值，代替 $c^{\langle t \rangle}$的值，tanh 激活函数计算，$W_c$ 权重，$c^{\langle t-1 \rangle}$ 前一个时间步的记忆值，$x^{\langle t \rangle}$ 当前输入值。$b_c)$ 偏移项。

	重点：GRU 中的真正思想。（决定是否更新）

- <font color=#0099ff>$Γ_u = \sigma(W_u [c^{\langle t-1 \rangle},x^{\langle t \rangle}] +  b_u)$

	$Γ_u$:更新门 （符号比较像 门的边缘），0 到 1 之间的值。$\sigma$ sigmoid 函数 （其函数图 回想下，介于 0 到 1 之间） u: update 更新 

	假设 cat 输出 是 $c^{\langle t \rangle} =1$ （1 代表单数，0 代表复数）memory cell 在从左往右读的过程中，会一直记住 这个值，直到 到了 was ，根据 $c^{\langle t \rangle} =1$ 所以决定用 was 。

	$Γ_u$ 决定什么时候更新这个值，比如 看到 the cat 那么这是个好的时机去更新  $c^{\langle t \rangle} =1$

- <font color=#0099ff>$c^{\langle t \rangle} = Γ_u * \tilde{c}^{\langle t \rangle} + (1- Γ_u) * c^{\langle t-1 \rangle}$

	$c^{\langle t \rangle} $ 等于 门的值 element-wise 候选值， $(1- Γ_u)$ element-wise  旧的值。若 $Γ_u ≈1$则代表把候选值 设为新的值。 若 $Γ_u ≈0$则代表 不更新，还是用旧的值。

详细的图解看上图 左侧图示。简化的 GRU 示意图。

优点：从左到右扫描句子时，门可以决定在哪个时间是否更新值。指导需要使用记忆细胞的时候。

 $Γ_u = 0.000000001$可以缓解梯度消失的问题，即使经过了很多很多的时间步，这个值也可以很好的维持。关键所在
 
注意细节： $c^{\langle t \rangle} $ 可以是向量，比如 维度 100 ，同理 上图中所标注出来的，都是相同的维度。

![这里写图片描述](http://img.blog.csdn.net/20180226160258111?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)


**完整 (Full) GRU :**

再添加一个门 $Γ_r $ ,r 可以理解为代表相关性，前一个 $c^{\langle t-1 \rangle}$ 对于计算 $\tilde{c}^{\langle t \rangle}$ 有多大的相关性。

$$\tilde{c}^{\langle t \rangle}==tanh (W_c [Γ_r  * c^{\langle t-1 \rangle},x^{\langle t \rangle}] +  b_c)$$

$$Γ_u = \sigma(W_u [c^{\langle t-1 \rangle},x^{\langle t \rangle}] +  b_u)$$

$$Γ_r = \sigma(W_r [c^{\langle t-1 \rangle},x^{\langle t \rangle}] +  b_r)$$

$$c^{\langle t \rangle} = Γ_u * \tilde{c}^{\langle t \rangle} + (1- Γ_u) * c^{\langle t-1 \rangle}$$

$$a^{\langle t \rangle} =c^{\langle t \rangle} $$


产生更大的影响，更好的缓解 梯度消失问题，GRU 是一个常用的版本。LSTM （Long Short Term Memory 长短时记忆网络）也是比较常用的。

---
## <font color=#0099ff>1.10 长短期记忆   Long Short Term Memory Unit （LSTM）
  
上一节学习的 GRU ，它能够在序列中学习到非常深的连接，此外 LSTM 也可以做到这一点。且比 GRU 更有效。

![这里写图片描述](http://img.blog.csdn.net/20180226172251822?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

**GRU 回顾：**

- $\tilde{c}^{\langle t \rangle}$: 代替记忆细胞的候选值。

- $Γ_u $: 更新门，决定什么时候更新。

- $Γ_r$:相关门，决定前面的序列对后面序列有多大的影响，彼此之间的相关性

- $c^{\langle t \rangle} $: 由上述几个值，共同决定是否更新。

- $a^{\langle t \rangle} =c^{\langle t \rangle} $: 这里是相等的。

**LSTM:**

$$\tilde{c}^{\langle t \rangle} = tanh (W_c [a^{\langle t-1 \rangle},x^{\langle t \rangle}] +  b_c)$$

(update) : $$Γ_u = \sigma(W_u [a^{\langle t-1 \rangle},x^{\langle t \rangle}] +  b_u)$$

(forget) :$$Γ_f = \sigma(W_f [a^{\langle t-1 \rangle},x^{\langle t \rangle}] +  b_f)$$

(output) :$$Γ_o = \sigma(W_o [a^{\langle t-1 \rangle},x^{\langle t \rangle}] +  b_o)$$

$$c^{\langle t \rangle} = Γ_u * \tilde{c}^{\langle t \rangle} +  Γ_f * c^{\langle t-1 \rangle}$$  # 是维持旧的记忆细胞的值，还是加上新的值。

$$a^{\langle t \rangle} =Γ_o * c^{\langle t \rangle} $$

在本周结束后的作业练习中：上面的公式是这么写的

$$ a^{\langle t \rangle} = \Gamma_o^{\langle t \rangle}* \tanh(c^{\langle t \rangle})$$

注意：这里是三个门，更新门，遗忘门，输出门。与 GRU 的区别，$a^{\langle t \rangle} $与$c^{\langle t \rangle} $不相等。


![这里写图片描述](http://img.blog.csdn.net/20180226172415589?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

公式其实更好理解，上右侧图 展示的更加直观些。

上图下方的红色直线，说明 只要正确了设置了 更新门和遗忘门，LSTM 就能把 $c^{\langle 0 \rangle} $ 一直往下传递到 后面某个记忆单元。这也是为什么 LSTM 和 GRU 善于维持更深的联系，及时经过很长很长的时间步。

此外，上述的 LSTM 与常用的 LSTM 单元 区别在于，

$Γ_o , Γ_f ,Γ_u $ 这些门值取决于，不仅是 $a^{\langle t-1 \rangle},x^{\langle t \rangle}$ 有时也可以 偷窥下 $c^{\langle t-1 \rangle}$的值，这叫做 窥视孔连接 （peephole connection），还有一个细节，如果 $c^{\langle t-1 \rangle}$ 如果是 一个 100 维的向量，其中 第 50 位是 1 ， 那么只会影响 第 50 位对应的门。

GRU 的优点：更简单，可以构建更深的网络，计算性能上，也运行的更快。
LSTM 优点：更加强大灵活，这是更好的选择。


---
## <font color=#0099ff>1.11 双向神经网络 （Bidirectional RNN）
    
到现在，之前已经学习了解了大部分 RNN 模型的关键的构建，此外还有两个方法，可以构建更好的模型，其中之一就是双向神经网络，这个模型可以让你在序列的某点处，不仅可以获取之前的信息，也可以获取未来的信息。本节讲解。还有一个是 Deep RNN 下节讲解。

![这里写图片描述](http://img.blog.csdn.net/20180226173329487?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

回顾之前的例子，命名实体识别问题（Named entity recognition），这个网络存在的一个问题是，当判断 Teddy 是否是人名中的一部分是，只看前面的部分是不够的，还需有后面的信息共同进行判断。需要使用 双向神经网络解决这个问题。


![这里写图片描述](http://img.blog.csdn.net/2018022617333950?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

**BRNN 工作原理：**

上图中，绿色画笔绘画的部分就是，反向连接，从最后依次反向向前连接，因此网络构成了一个无环图（Acyclic graph）。

反向序列从 $\xleftarrow[a]{}^{\langle 4 \rangle}$,反向进行，到$\xleftarrow[a]{}^{\langle 3 \rangle}$.....

所以整个前向广播，分为两部分，一部分是从左到右，另一部分是从右到左。

最终预测输出的值：$\hat{y}^{\langle t \rangle} = g (W_y [\vec{a}^{\langle t \rangle},\xleftarrow[a]{}^{\langle t \rangle}] +  b_y)$

(没有找到合适的 latex 在字母上方表示向左的箭头)

实践中，大部分的 NLP 问题，采用 BRNN W/LSTM  (有 LSTM 单元的 BRNN 模型)是一个不错的选择。

BRNN 缺点：需要完整的数据的序列，才能预测任意位置。比如，构建语音识别系统，需要等待一个人说完整个句子，才能语音处理，并进一步识别，所以实际上的大型的应用，都是更加复杂的模型。

---
## <font color=#0099ff>1.12 深层循环神经网络 (Deep RNNs)
    
目前为止，学习到的不同的 RNN 版本，每一个都可以独当一面，但是要学习非常复杂的函数，通常的做法是把 RNN 的多个层堆叠起来，来构建更深的模型。

目标：学习如何构建更深的模型。


![这里写图片描述](http://img.blog.csdn.net/20180226173347938?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)    
    
与之前的 标准 （standard）RNN 相比，改变了下符号表示，将 $a^{\langle 0 \rangle}$ 改为 $a^{[1]<0 \rangle}$ 

$a^{[l]<t \rangle}$:$l$ 表示 $l$ 层。 t : 第 t 个时间步。

上图所示为，三个隐层的新网络。

其中$a^{[2]<3 \rangle}$ 的计算是：

$$a^{[2]<3 \rangle} = g(W^{[2]}_a [a^{[2]<2 \rangle},a^{[1]<3 \rangle}] +  b^{[2]}_a)$$


$y^{\langle 1 \rangle}$ 输出之前可以做的改变是，增加纵向的连接，但是横向的连接不会增加。这种结构更加常见。

其中的单元，可以是 标准的 RNN ,也可以是 GRU ,或者 LSTM BRNN 。
    
---
 
**PS: 欢迎扫码关注公众号：「SelfImprovementLab」！专注「深度学习」，「机器学习」，「人工智能」。以及 「早起」，「阅读」，「运动」，「英语 」「其他」不定期建群 打卡互助活动。**

<center><img src="http://upload-images.jianshu.io/upload_images/1157146-cab5ba89dfeeec4b.jpg?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240"></center>